In [ ]:
import tensorflow_datasets as tfds
from tqdm import tqdm
import wikipedia
wiki_ds = tfds.load('wiki40b/ja', split='test')

In [ ]:
def get_wikipedia_title(split_sentence_list):
  this_process = True
  while this_process:
    if split_sentence_list[0] == '':
      split_sentence_list = split_sentence_list[1:]
    elif 'START_ARTICLE' in split_sentence_list[0]:
      this_process = False
  title = split_sentence_list[split_sentence_list.index('_START_ARTICLE_') + 1]
  return title

def get_wikipedia_summary(split_sentence_list):
  try:
    this_process = True
    while this_process:
      if split_sentence_list[0] == '':
        split_sentence_list = split_sentence_list[1:]
      elif 'START_ARTICLE' in split_sentence_list[0]:
        this_process = False

    title = split_sentence_list[split_sentence_list.index('_START_ARTICLE_') + 1]

    wikipedia.set_lang("jp")
    # テキストファイルを開く

    # 検索ワードを用いて検索
    words = wikipedia.search(title)

    if not words:
        raise ValueError("no word")
    else:
        #検索ワードがヒットすれば要約を取得
        summary = str(wikipedia.summary(words[0])).replace('\n', '')

    return title, summary

  except Exception as e:
    print(title)
    return None, None

In [ ]:
# titleのみ取得して出力
title_list = []
for wiki_content in tqdm(list(wiki_ds.as_numpy_iterator())):
  title = get_wikipedia_title(wiki_content['text'].decode().split('\n'))
  title_list.append(title)

wikipedia_title_dict = {'title':title_list}
import json
path = '/content/drive/MyDrive/work/detection_ai_generated_sentence/wiki_title_dict.json'
json_file = open(path, mode="w")
json.dump(wikipedia_title_dict, json_file, indent=2, ensure_ascii=False)
json_file.close()

In [ ]:
# titleからsummaryを取得
wiki_contents_dict = {}
for wiki_content in tqdm(list(wiki_ds.as_numpy_iterator())):
  title, summary = get_wikipedia_summary(wiki_content['text'].decode().split('\n'))
  if title==None or summary==None:
    pass
  else:
    if '_NEWLINE_' in summary:
      summary = ''.join(summary.split('_NEWLINE_'))
    wiki_contents_dict[title] = summary

In [ ]:
import json
path = '/content/drive/MyDrive/work/detection_ai_generated_sentence/wiki_contents_dict.json'
json_file = open(path, mode="w")
json.dump(wiki_contents_dict, json_file, indent=2, ensure_ascii=False)
json_file.close()